In [78]:
import tensorflow_datasets as tfds
import tensorflow as tf

import matplotlib.pyplot as plt

dataset = tfds.load( name="cifar10", split="train")
dataset = dataset.shuffle( 1024 ).batch( 62 )

#(xTrainFull, yTrainFull), (xTest, yTest) = tfds.load( name="cifar10" , split=tfds.Split.TRAIN )

#xValid, xTrain = xTrainFull[:5000] / 255.0, xTrainFull[5000:] / 255.0
#yValid, yTrain = yTrainFull[:5000], yTrainFull[5000:]

output_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [79]:
leaky_relu_alpha = 0.2
dropout_rate = 0.5

def conv2d( inputs , filters , stride_size ):
    out = tf.nn.conv2d( inputs , filters , strides=[ 1 , stride_size , stride_size , 1 ] , padding="SAME" ) 
    return tf.nn.leaky_relu( out , alpha=leaky_relu_alpha ) 

def maxpool( inputs , pool_size , stride_size ):
    return tf.nn.max_pool2d( inputs , ksize=[ 1 , pool_size , pool_size , 1 ] , padding='VALID' , strides=[ 1 , stride_size , stride_size , 1 ] )

def dense( inputs , weights ):
    x = tf.nn.leaky_relu( tf.matmul( inputs , weights ) , alpha=leaky_relu_alpha )
    return tf.nn.dropout( x , rate=dropout_rate )

In [93]:
initializer = tf.initializers.glorot_uniform()
def get_weight( shape , name ):
    return tf.Variable( initializer( shape ) , name=name , trainable=True , dtype=tf.float32 )

shapes = [
    [ 3 , 3 , 3 , 16 ] , 
    [ 3 , 3 , 16 , 16 ] , 
    [ 3 , 3 , 16 , 32 ] , 
    [ 3 , 3 , 32 , 32 ] ,
    [ 2048 , 1600 ] , 
    [ 1600 , 64 ] ,
    [ 64 , 3 ]
]

weights = []
for i in range( len( shapes ) ):
    weights.append( get_weight( shapes[ i ] , 'weight{}'.format( i ) ) )
    

In [81]:
def model( x ) :
    x = tf.cast( x , dtype=tf.float32 )
    c1 = conv2d( x , weights[ 0 ] , stride_size=1 ) 
    c1 = conv2d( c1 , weights[ 1 ] , stride_size=1 ) 
    p1 = maxpool( c1 , pool_size=2 , stride_size=2 )
    
    c2 = conv2d( p1 , weights[ 2 ] , stride_size=1 )
    c2 = conv2d( c2 , weights[ 3 ] , stride_size=1 ) 
    p2 = maxpool( c2 , pool_size=2 , stride_size=2 )

    flatten = tf.reshape( p2 , shape=( tf.shape( p2 )[0] , -1 ))
    
    d1 = dense( flatten , weights[ 4 ] )
    d2 = dense( d1 , weights[ 5 ] )
    logits = tf.matmul( d2 , weights[ 6 ] )

    return tf.nn.softmax( logits )

In [82]:
def loss( pred , target ):
    return tf.losses.categorical_crossentropy( target , pred )

In [86]:
learning_rate = 1
optimizer = tf.optimizers.SGD( learning_rate )


def train_step( model, inputs , outputs ):
    with tf.GradientTape() as tape:
        current_loss = loss( model( inputs ), outputs)
    grads = tape.gradient( current_loss , weights )
    optimizer.apply_gradients( zip( grads , weights ) )
    print( tf.reduce_mean( current_loss ) )
    
num_epochs = 1
for e in range( num_epochs ):
    for features in dataset:
        image , label = features[ 'image' ] , features[ 'label' ]
        train_step( model , image , tf.one_hot( label , depth=3 ) )

tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(4.1595087, shape=(), dtype=float32)
tf.Tensor(3.6395702, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(1.819785, shape=(), dtype=float32)
tf.Tensor(3.8995395, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(4.939417, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(1.819785, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(4.939417, shape=(), dtype=float32)
tf.Tensor(3.8995395, shape=(), dtype=float32)
tf.Tensor(2.599693, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.8995392, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float

tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(1.819785, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(4.1595087, shape=(), dtype=float32)
tf.Tensor(2.599693, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(1.819785, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float32)
tf.Tensor(4.679447, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.6395702, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)


tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(2.8596623, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float32)
tf.Tensor(2.599693, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(5.979294, shape=(), dtype=float32)
tf.Tensor(3.8995392, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.6395702, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)


tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.6395702, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(4.1595087, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.3796008, shape=(), dtype=float32)
tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(4.1595087, shape=(), dtype=float32)
tf.Tensor(1.819785, shape=(), dtype=float32)
tf.Tensor(2.0797544, shape=(), dtype=float32)
tf.Tensor(2.8596623, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(3.899539, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)

tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(4.939417, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float32)
tf.Tensor(3.63957, shape=(), dtype=float32)
tf.Tensor(4.419478, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(2.3397236, shape=(), dtype=float32)
tf.Tensor(2.599693, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(2.859662, shape=(), dtype=float32)
tf.Tensor(5.719324, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float32)
tf.Tensor(2.0797544, shape=(), dtype=float32)
tf.Tensor(3.3796005, shape=(), dtype=float32)
tf.Tensor(4.9394164, shape=(), dtype=float32)
tf.Tensor(3.8995392, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=float32)
tf.Tensor(2.5996928, shape=(), dtype=float32)
tf.Tensor(3.6395702, shape=(), dtype=float32)
tf.Tensor(3.1196313, shape=(), dtype=floa

In [101]:
import numpy as np

testSet = tfds.load( name="cifar10", split="test")
testSet = testSet.shuffle( 1024 ).batch( 62 )
for i, feature in enumerate(testSet):
    image, label = feature['image'], feature['label']
    prediction = model(image)
    shit = np.where(label==1)
    print(shit)
    print(str(i) + " : " + str(loss(model(image), label)))

(array([ 2, 13, 17, 25, 34, 51, 55], dtype=int64),)


InvalidArgumentError: Incompatible shapes: [62] vs. [62,3] [Op:Mul] name: mul/